In [2]:
import numpy as np
import torch

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer

student_model = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(student_model)

dataset = load_dataset("imdb")

def pre_process(examples):
    return tokenizer(examples["text"], truncation = True, max_length = 512)

tokenized_data = dataset.map(pre_process, batched = True)


Map: 100%|██████████| 50000/50000 [00:08<00:00, 5572.51 examples/s]


In [12]:
print(tokenized_data)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [13]:
from transformers import TrainingArguments, Trainer
import torch
import torch.nn as nn
import torch.nn.functional as F

2024-06-05 12:28:16.618086: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
class distillTrainer(Trainer):
    def __init__(self, *args, teacher_model = None, temperature = None, **kwargs):
        super().__init__(*args,**kwargs)
        self.teacher = teacher_model
        self.temperature = temperature
        self.device = if torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.teacher.to(self.device)
        self.teacher.eval()

    def compute_loss(self, model, inputs, return_outputs = False):
        #Distillation loss over soft target probabilities of teacher and student
        #Cosine distance loss
        #Language modelling loss
        
        s_outputs = model(**inputs)
        t_outputs = self.teacher(**inputs)
        #soft target probabilities
        soft_student = F.log_softmax(s_outputs.logits / self.temperature, dim = 1)
        soft_teacher = F.softmax(t_outputs.logits / self.temperature, dim = 1)
        
        
         
        